In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator

2022-11-21 05:32:20.950328: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Homework

### Dataset

In this homework, we'll build a model for predicting if we have an image of a dino or a dragon. For this, we will use the "Dino or Dragon?" dataset that can be downloaded from [Kaggle](https://www.kaggle.com/datasets/agrigorev/dino-or-dragon). 

You can get a wget-able version [here](https://github.com/alexeygrigorev/dino-or-dragon/releases/download/data/dino-dragon.zip):

```bash
wget https://github.com/alexeygrigorev/dino-or-dragon/releases/download/data/dino-dragon.zip
unzip dino-dragon.zip
```

In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch. 

**Note:** You will need an environment with a GPU for this homework. We recommend to use [Saturn Cloud](https://bit.ly/saturn-mlzoomcamp). 
You can also use a computer without a GPU (e.g. your laptop), but it will be slower.

In [2]:
import os

In [3]:
if not os.path.isfile('dino-dragon.zip'):
    !wget https://github.com/alexeygrigorev/dino-or-dragon/releases/download/data/dino-dragon.zip
    !unzip dino-dragon.zip

### Data Preparation

The dataset contains around 1900 images of dinos and around 1900 images of dragons. 

The dataset contains separate folders for training and test sets. 

### Model

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(150, 150, 3)`
* Next, create a convolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`

For clarification about kernel size and max pooling, check [Office Hours](https://www.youtube.com/watch?v=1WRgdBTUaAc).

In [4]:
optimizer = keras.optimizers.SGD(learning_rate=0.002, momentum=0.8)
loss = keras.losses.binary_crossentropy
input_shape = (150, 150, 3)

In [5]:
inputs = keras.Input(shape=input_shape)

conv = keras.layers.Conv2D(
    filters=32,
    kernel_size=(3, 3),
    activation='relu'
)(inputs)

pool = keras.layers.MaxPooling2D(
    pool_size=(2, 2)
)(conv)

flat = keras.layers.Flatten()(pool)
dense = keras.layers.Dense(64, activation='relu')(flat)
outputs = keras.layers.Dense(1, activation=tf.nn.sigmoid)(dense)

model = keras.Model(inputs, outputs)

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['accuracy']
)

2022-11-21 05:32:22.819618: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 05:32:22.826978: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 05:32:22.827650: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-21 05:32:22.828653: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

### Question 1

Since we have a binary classification problem, what is the best loss function for us?

- `binary crossentropy`
- `focal loss`
- `mean squared error`
- `categorical crossentropy`

Note: since we specify an activation for the output layer, we don't need to set `from_logits=True`

* Binary Crossentropy (focal for cass imbalance, MSE for regression, categorical crossentropy for muti-class)

### Question 2

What's the total number of parameters of the model? You can use the `summary` method for that. 

- 9215873
- 11215873
- 14215873
- 19215873

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                             

### Generators and Training

For the next two questions, use the following data generator for both train and test sets:

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/val directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and test sets. 

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)
```

In [9]:
train_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_ds = train_gen.flow_from_directory(
    'train/',
    class_mode='binary',
    target_size=(150, 150),
    batch_size=20,
    shuffle=True
)

test_ds = train_gen.flow_from_directory(
    'test/',
    class_mode='binary',
    target_size=(150, 150),
    batch_size=20,
    shuffle=True
)

Found 1594 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


In [10]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds
)

Epoch 1/10


2022-11-21 05:32:35.264874: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-11-21 05:32:35.914508: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-21 05:32:35.915385: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-21 05:32:35.915428: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-11-21 05:32:35.916301: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-21 05:32:35.916383: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


80/80 [==============================] - 11s 109ms/step - loss: 0.5782 - accuracy: 0.6863 - val_loss: 0.4531 - val_accuracy: 0.8122
Epoch 2/10
80/80 [==============================] - 9s 106ms/step - loss: 0.4333 - accuracy: 0.8080 - val_loss: 0.3787 - val_accuracy: 0.8528
Epoch 3/10
80/80 [==============================] - 8s 106ms/step - loss: 0.3854 - accuracy: 0.8369 - val_loss: 0.3685 - val_accuracy: 0.8401
Epoch 4/10
80/80 [==============================] - 8s 106ms/step - loss: 0.3317 - accuracy: 0.8645 - val_loss: 0.3390 - val_accuracy: 0.8528
Epoch 5/10
80/80 [==============================] - 9s 106ms/step - loss: 0.2902 - accuracy: 0.8908 - val_loss: 0.3368 - val_accuracy: 0.8503
Epoch 6/10
80/80 [==============================] - 8s 105ms/step - loss: 0.2647 - accuracy: 0.8965 - val_loss: 0.3507 - val_accuracy: 0.8376
Epoch 7/10
80/80 [==============================] - 8s 106ms/step - loss: 0.2353 - accuracy: 0.9090 - val_loss: 0.2987 - val_accuracy: 0.8680
Epoch 8/10
80/80

### Question 3

What is the median of training accuracy for all the epochs for this model?

- 0.40
- 0.60
- 0.90
- 0.20

In [11]:
np.median(history.history['accuracy'])

0.8936637341976166

### Question 4

What is the standard deviation of training loss for all the epochs for this model?

- 0.11
- 0.66
- 0.99
- 0.33

In [18]:
np.std(history.history['loss'])

0.12144276344543038

### Data Augmentation

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=40,`
* `width_shift_range=0.2,`
* `height_shift_range=0.2,`
* `shear_range=0.2,`
* `zoom_range=0.2,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`

In [21]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_ds = train_gen.flow_from_directory(
    'train/',
    class_mode='binary',
    target_size=(150, 150),
    batch_size=20,
    shuffle=True
)

Found 1594 images belonging to 2 classes.


In [22]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds
)

Epoch 1/10
80/80 [==============================] - 15s 190ms/step - loss: 0.4787 - accuracy: 0.7792 - val_loss: 0.4976 - val_accuracy: 0.7665
Epoch 2/10
80/80 [==============================] - 15s 190ms/step - loss: 0.4297 - accuracy: 0.7986 - val_loss: 0.2744 - val_accuracy: 0.8782
Epoch 3/10
80/80 [==============================] - 15s 190ms/step - loss: 0.4110 - accuracy: 0.8162 - val_loss: 0.3424 - val_accuracy: 0.8528
Epoch 4/10
80/80 [==============================] - 15s 188ms/step - loss: 0.3939 - accuracy: 0.8193 - val_loss: 0.2993 - val_accuracy: 0.8680
Epoch 5/10
80/80 [==============================] - 15s 189ms/step - loss: 0.3826 - accuracy: 0.8237 - val_loss: 0.3553 - val_accuracy: 0.8477
Epoch 6/10
80/80 [==============================] - 15s 188ms/step - loss: 0.3691 - accuracy: 0.8331 - val_loss: 0.5256 - val_accuracy: 0.7766
Epoch 7/10
80/80 [==============================] - 15s 188ms/step - loss: 0.3719 - accuracy: 0.8388 - val_loss: 0.4265 - val_accuracy: 0.8249

### Question 5 

Let's train our model for 10 more epochs using the same code as previously.
Make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

- 0.15
- 0.77
- 0.37
- 0.97

In [23]:
np.mean(history.history['val_loss'])

0.39190123677253724

### Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

- 0.84
- 0.54
- 0.44
- 0.24

In [24]:
np.mean(history.history['val_accuracy'])

0.8314720809459686

## Submit the results

- Submit your results here: https://forms.gle/XdH5ztBddvTvxzpT6
- You can submit your solution multiple times. In this case, only the last submission will be used
- If your answer doesn't match options exactly, select the closest one

## Deadline

The deadline for submitting is 21 November 2022, 23:00 CEST.

After that, the form will be closed.